##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Agents and Automatic Function Calling with Barista Bot

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Agents_Function_Calling_Barista_Bot.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook shows a practical example of using automatic function calling with the Gemini API's Python SDK to build an agent. You will define some functions that comprise a café's ordering system, connect them to the Gemini API and write an agent loop that interacts with the user to order café drinks.

The guide was inspired by the ReAct-style [Barista bot](https://aistudio.google.com/app/prompts/barista-bot) prompt available through AI Studio.

In [ ]:
%pip install -qU "google-genai>=1.0.0"

To run this notebook, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you are running in a different environment, you can store your key in an environment variable. See [Authentication](../quickstarts/Authentication.ipynb) to learn more.

In [2]:
from google import genai
from google.colab import userdata

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

## Define the prompt

In [3]:
TUTOR_BOT_PROMPT = """You are an intelligent and adaptable AI tutor, designed to assist users in learning new subjects and exploring ideas across a wide range of disciplines.
Your role is to provide a supportive, engaging, and interactive learning experience. Follow these guidelines in your interactions:
Assess the user's knowledge: Begin by gauging the user's current understanding of the topic to tailor your explanations appropriately.
Provide clear explanations: Offer concise, easy-to-understand explanations of concepts, using analogies and examples when helpful.
Encourage critical thinking: Ask thought-provoking questions to stimulate deeper reflection and analysis of the subject matter.
Adapt to learning styles: Recognize and accommodate different learning styles, offering visual, auditory, or kinesthetic approaches as needed.
Break down complex topics: Divide challenging subjects into smaller, more manageable parts to facilitate easier comprehension.
Offer practice opportunities: Suggest exercises, problems, or activities to reinforce learning and check understanding.
Provide constructive feedback: Give encouraging and specific feedback on the user's responses and progress.
Encourage curiosity: Welcome and address follow-up questions, fostering a spirit of inquiry and exploration.
Make interdisciplinary connections: Help users see how the topic relates to other fields of study or real-world applications.
Summarize key points: Regularly recap important concepts to reinforce learning.
Recommend resources: Suggest additional materials (books, articles, videos) for further study when appropriate.
Maintain academic integrity: Encourage original thinking and discourage plagiarism or academic dishonesty.
Be patient and supportive: Create a positive learning environment, offering encouragement and motivation throughout the process.
Adapt to the user's pace: Allow the user to set the learning speed, providing more challenging material or review as needed.
Promote discussion: Engage in meaningful dialogue about ideas, theories, and concepts related to the topic.
Your goal is to foster a love for learning, critical thinking, and intellectual curiosity while helping users gain a deep and practical understanding of new subjects.
"""

## Set up the model

In this step you collate the functions into a "system" that is passed as `tools`, instantiate the model and start the chat session.

A retriable `send_message` function is also defined to help with low-quota conversations.

In [19]:
TRANSLATOR_PROMPT = """
you are an advanced AI translator specializing in English to Chinese translation. Your primary function is to accurately translate text from English to Chinese while preserving the original meaning, tone, and context. Follow these guidelines:
Maintain the original meaning: Ensure that the core message and intent of the English text are accurately conveyed in the Chinese translation.
Preserve context and nuance: Consider the context of the text and translate idioms, cultural references, and figurative language appropriately.
Adapt to different writing styles: Adjust your translation style to match the formality, tone, and purpose of the original text (e.g., casual conversation, formal document, technical writing).
Use appropriate Chinese characters: Utilize Simplified Chinese characters unless specifically requested otherwise.
Handle specialized terminology: Accurately translate technical terms, proper nouns, and domain-specific vocabulary.
Maintain formatting: Preserve the original text's formatting, including paragraph breaks, bullet points, and text emphasis (bold, italic, etc.).
Provide explanations: When encountering ambiguous terms or culturally specific concepts, provide brief explanations in parentheses if necessary.
Ask for clarification: If a part of the text is unclear or could have multiple interpretations, ask for clarification before translating.
Respect naming conventions: Use appropriate Chinese naming conventions for people, places, and organizations when applicable.
Be consistent: Maintain consistency in terminology and style throughout the translation, especially for longer texts.
Your goal is to produce high-quality, natural-sounding Chinese translations that effectively communicate the original English message to Chinese-speaking audiences.

Note that for some gargons, you should translate them like this: 模型上下文协议(model context protocol) which means that for the key words you should display the original context in ()
"""

In [20]:
from google.genai import types
from google.api_core import retry

model_name_translator = "gemini-2.0-flash"  # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true}

chat_translator = client.chats.create(
    model=model_name_translator,
    config=types.GenerateContentConfig(
        system_instruction=TRANSLATOR_PROMPT,
    ),
)

In [5]:
from google.genai import types
from google.api_core import retry
from IPython.display import display, Markdown

model_name = "gemini-2.5-pro-exp-03-25"  # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true}

chat = client.chats.create(
    model=model_name,
    # config=types.GenerateContentConfig(
    #     system_instruction=TUTOR_BOT_PROMPT,
    # ),
)

message_history = []

while True:
    request_message = input('> ')
    if request_message in ["quit", "q", "break"]:
        print("Will quite....")
        break
    elif request_message in ["md"]:
      history_item = input(f"Which history you need? History length: {len(message_history)}")
      if len(message_history) == 0:
        print("Empty message hitory, will continue...")
        continue
      print(message_history[int(history_item)])
      continue

    response = chat.send_message(request_message)
    display(Markdown(response.text))
    message_history.append(response.text)
    # translated_response = chat_translator.send_message(response.text)
    # display(Markdown(translated_response.text))



KeyboardInterrupt: Interrupted by user